# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from IPython.display import display

In [16]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [11]:
# Create our features
# YOUR CODE HERE
X = df.copy()

# Binary encode the string columns to int columns
X = pd.get_dummies(X, columns=['issue_d', 'verification_status', 'home_ownership', 'hardship_flag', 'debt_settlement_flag', 'pymnt_plan', 'initial_list_status', 'application_type','next_pymnt_d'])

# Drop the unecessary columns
X.drop('loan_status', axis=1, inplace=True)

# Create our target
# YOUR CODE HERE
y = df['loan_status'].to_frame()


In [6]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,1,1,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,0,1,1,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,0,1,1,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,1,1,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,1,1,1,0,1,1,0


In [7]:
# Check the balance of our target values
# YOUR CODE HERE
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(
    X
    , y
    , random_state=47
    , stratify=y
)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
# YOUR CODE HERE
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE

X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [20]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
balanced_rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=78)
balanced_rf_model = balanced_rf_model.fit(X_train_scaled, y_train)

balanced_rf_pred = balanced_rf_model.predict(X_test_scaled)

In [22]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
accuracy_score = balanced_accuracy_score(y_test, balanced_rf_pred)
accuracy_score

0.731128958829383

In [25]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, balanced_rf_pred)
cm_df = pd.DataFrame(
    cm
    , index=['Actual 0', 'Actual 1']
    , columns=['Predicted 0', 'Predicted 1']
)
cm_df_title = 'Confusion Matrix'

In [27]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(cm_df_title)
display(cm_df)
print(f"Accuracy Score: {accuracy_score.round(2)}")
print('Classification Report')
print(classification_report_imbalanced(y_test, balanced_rf_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,50,37
Actual 1,1925,15193


Accuracy Score: 0.73
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.57      0.89      0.05      0.71      0.49        87
   low_risk       1.00      0.89      0.57      0.94      0.71      0.53     17118

avg / total       0.99      0.89      0.58      0.93      0.71      0.53     17205



In [29]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
sorted(zip(balanced_rf_model.feature_importances_, X.columns), reverse=True)

[(0.06638784722481922, 'total_pymnt'),
 (0.06543489695017614, 'total_rec_int'),
 (0.06239824564201449, 'last_pymnt_amnt'),
 (0.0593164988668372, 'total_pymnt_inv'),
 (0.05747153782487019, 'total_rec_prncp'),
 (0.03283966480335606, 'int_rate'),
 (0.017977599737635526, 'issue_d_Jan-2019'),
 (0.017935743228689103, 'tot_hi_cred_lim'),
 (0.016865462366065877, 'out_prncp_inv'),
 (0.016400573476249587, 'total_rev_hi_lim'),
 (0.015994236427120626, 'avg_cur_bal'),
 (0.014900723751443816, 'max_bal_bc'),
 (0.014727681385274688, 'mo_sin_old_rev_tl_op'),
 (0.014723009884902612, 'revol_bal'),
 (0.014677282996722513, 'installment'),
 (0.014223544872656863, 'mths_since_recent_bc'),
 (0.014124129539338994, 'bc_open_to_buy'),
 (0.014105075117852802, 'out_prncp'),
 (0.01389284861939439, 'dti'),
 (0.01381546304171573, 'total_bal_ex_mort'),
 (0.013288371382091255, 'annual_inc'),
 (0.013215265901882623, 'mo_sin_old_il_acct'),
 (0.013102801349388607, 'all_util'),
 (0.012858040844688818, 'total_bal_il'),
 (0.

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.